# Homework (deadline 23.12.2022 23:59:59)

Write solutions for the homework exercises in this notebook. Once the work is done download the notebook file (`File > Download .ipynb`) rename it properly so it follows a template `HW2_<SURNAME>_<NAME>.ipynb` and send the file to me. **Please also attach all files you will be asked to produce in this homework**. My email address is as follows: 

* <m.biesaga@uw.edu.pl>

Remember that you can contact me via email if you have any problems. Moreover, you can also visit me in the ISS on the fourth floor (room 415). Usually, I am there from 11ish but please let me know in advance if you are coming because I might be busy. 


## Task 1 (15 points)
You all should be able to get the file `links.jl` on our [Google Drive](https://drive.google.com/file/d/11m7TWX4w1T5UfhvPzTrLFlU-wOhWc5A0/view?usp=sharing). It contains 30 links to EURACTIV, The Guardian, and The Parliament Magazine articles. Your task is to:

* Read `links.jl` to Python (Google Colab)
* Scrap the articles from the given links
* Write out the results into `HW2_<SURNAME>_<NAME>.jl`

From every website you should get the following information:

* Title of the article
* Author of the text (if present, if not the value should be None)
* Lead of the article (summary)
* Date of publication
* Text of the article

Additionally, you should count the number of words in the given article and extract only relevant information from the `links.jl`.

*Reminder:* A JSON line file contains multiple JSONs. As the name suggests each line contains one. Therefore, each line should look more or less like that:

```python
{ 
	"title" : "Why are Namedays better than Birthdays?",
    "author" : "M. Biesaga",
    "date" : "06.12.2019",
    "lead" : "Scientists from one of the best Universities in the U.S. proved that the discussion about birthdays and namedays is finally over.",
    "content" : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Integer eget sapien nisi. In placerat nisl felis, vel porttitor odio aliquam quis. Nulla a facilisis arcu. Suspendisse potenti. Interdum et malesuada fames ac ante ipsum primis in faucibus. Duis et semper urna. Mauris sit amet enim ex. Integer eget ultricies enim, at tristique sem. In eu eros nisl. Nulla vitae pretium risus, quis vestibulum dui. Nullam vitae dapibus quam. Maecenas commodo dictum ex, id vestibulum ex volutpat interdum. Cras tempor diam non urna auctor, vitae dignissim tortor tristique. Nunc consectetur mauris non lorem luctus aliquam. Mauris vitae ligula orci.",
	"source" : "Journal of Scientific Science",
	"fb" : { 
		        "likes" : 112,
	            "shares" : 2,
			    "comments" : 43
	},
	"length" : 98 
}
```
**IMPORTANT**: If you are going to send requests in a loop please don't send more than 1 request per second. You can use `time.sleep(1)` from the `time` module to halt your code for 1 second. For example, in a loop it would look something like that:
```python
for line in links:
	## Send request
	time.sleep(1)
```
It will perform the code in a loop and at the end of each iteration, it will wait for a second.

In [ ]:
import json
import requests as rq
from bs4 import BeautifulSoup
import time

## YOUR CODE

## Solution 1

In [ ]:
with open('links.jl', 'r') as file:
    df = [ json.loads(line) for line in file.readlines() ]

In [ ]:
output = []

for line in df:
  temp_dict ={}
  temp_dict['source_name'] = line['source_name']
  temp_dict['fb'] = {}
  temp_dict['fb']['likes'] = line['fb']['likes']
  temp_dict['fb']['shares'] = line['fb']['shares']
  temp_dict['fb']['comments'] = line['fb']['comments']
  
  response = rq.get(line['url'])
  
  html = BeautifulSoup(response.content, 'html.parser')


  if temp_dict['source_name'] == 'EURACTIV':
    temp_dict['title'] = html.select_one('section.ea-post-title').text.strip()
	  ## I don't extract text from the HTML object because I wll do it later for
    ## all sources.
    temp_dict['author'] = html.select_one('a.author')
    temp_dict['lead'] = ''
    temp_dict['date'] = html.select_one('span.ea-dateformat').text.strip()
	  ## Extract the content of the article in the list comprehension.
    temp_dict['content'] = ' '.join( p.text.strip() for p in html.select('article div.ea-article-body-content p') )
  elif temp_dict['source_name'] == 'The Guardian':
    temp_dict['title'] = html.select_one('article div h1').text.strip()
    ## I don't extract text from the HTML object because I wll do it later for
    ## all sources. In The Guardian, one article would return None because
    ## apparently it does not have an author. If I used the strip() method on
    ## the None value it would return an error.
    temp_dict['author'] = html.select_one('address div a')
    temp_dict['date'] = html.select_one('details summary span').text.strip()
    temp_dict['lead'] = html.select_one('div p').text.strip()
    temp_dict['content'] = ' '.join( p.text.strip() for p in html.select('article div#maincontent p') )
  else:
    temp_dict['title'] = html.select_one('div.av-title h1').text.strip()
    ## I don't extract text from the HTML object because I will do it later for
    ## all sources.
    temp_dict['author'] = html.select_one('main p.av-authName a')
    temp_dict['date'] = html.select_one('div.av-authInfo p.av-date').text.strip()
    temp_dict['lead'] = html.select_one('div.av-standFirst.playfair').text.strip()
    temp_dict['content'] = ' '.join( p.text.strip() for p in html.select('div.container div.av-main p') )
    ## Here, I check whether the author field is None. If not
    ## I extract the name of the author.
  if temp_dict['author'] is not None:
    temp_dict['author'] = temp_dict['author'].text.strip()
  ## Compute the length of the article.
  temp_dict['length'] = len(temp_dict['content'].split())

  output.append(temp_dict)
  time.sleep(1)

In [ ]:
with open('HW2_Biesaga_Mikolaj.jl', 'w') as file:
    for line in output:
        file.write( json.dumps(line) + '\n' )

## Solution 2

In [ ]:
## Open connection with the output file.
with open('HW2_Biesaga_Mikolaj.jl', 'w') as write_file:
    ## Open connection with the file with links.
    with open('links.jl', 'r') as read_file:
        ## Iterate over every single line of the file.  # Note that this method
        ## reads a single line at the time. So I never have more than one line
        ## in the working memory. Unlike file.readlines() that stores all lines
        ## in the working memory.
        for line in read_file:
            ## Convert a string into a dictionary.
            line = json.loads(line)
            ## Create an empty dictionary.
            temp_dict = {}
            ## Send a request and convert it to HTML code.
            html = BeautifulSoup(rq.get(line['url']).content, 'html.parser')
            ## Extract the title of the articles. I've noticed that all pages
            ## have the titles in the head under meta tag. The only difference
            ## between web pages is that the attribute sometimes is called name
            ## other times property. Therefore I use this vertical line that
            ## indicates or. In other words, in this line, I am looking for a tag
            ## called meta with either name or property attribute equal to
            ## "og:title". In both cases the title of the article is the value of
            ## the content attribute.
            temp_dict['title'] = html.select_one('meta[name|property="og:title"]').get('content')
            ## Extract the description of the article. It is a similar case to
            ## extracting the title.
            temp_dict['lead'] = html.select_one('meta[name|property="og:description"]').get('content')
            ## Extract the published date. This time it is easier because the
            ## name of the attribute is always the same.
            temp_dict['published_at'] = html.select_one('meta[property="article:published_time"]').get('content')
            ## Copy the name of the webpage from the links file.
            temp_dict['source_name'] = line['source_name']
            ## Both the author field and content field for these three sources
            ## will have different tags. Therefore, I use the if-statement.
            if temp_dict['source_name'] == 'The Guardian':
                ## In that case, there is no point for me to extract from the
                ## HTML code the text inside the if-statement because this field
                ## might be empty. I will deal with it later.
                temp_dict['author'] = html.select_one('main a[rel="author"]')
                ## Extract the content of the webpage and convert it into a string. 
                temp_dict['content'] = ' '.join( p.text.strip() for p in html.select('article div#maincontent p') )
            elif temp_dict['source_name'] == 'EURACTIV':
                ## In that case, there is no point for me to extract from the
                ## HTML code the text inside the if-statement because this field
                ## might be empty. I will deal with it later.
                temp_dict['author'] = html.select_one('article a.author')
                ## Extract the content of the webpage and convert it into a string. 
                temp_dict['content'] = ' '.join( p.text.strip() for p in html.select('article div.ea-article-body-content p') )
            else:
                ## In that case, there is no point for me to extract from the
                ## HTML code the text inside the if-statement because this field
                ## might be empty. I will deal with it later.
                temp_dict['author'] = html.select_one('main p.av-authName a')
                ## Extract the content of the webpage and convert it into a string. 
                temp_dict['content'] = ' '.join( p.text.strip() for p in html.select('div.container div.av-main p') )
    
    		## Compute the length of the article. It is not a perfect solution
    		## but it is good enough for our case.
            temp_dict['length'] = len(temp_dict['content'].split())
            ## Check whether the author field is not None
            if temp_dict['author'] is not None:
                ## Extract from the HTML code the name of the author
                temp_dict['author'] = temp_dict['author'].text.strip()  
            ## Extract data about Facebook likes
            temp_dict['fb'] = { 'likes' : line['fb']['likes'],
                                 'shares' : line['fb']['shares'],
                                 'comments' : line['fb']['comments']
            }
            ## Write out the data to the file
            write_file.write(json.dumps(temp_dict) + '\n')
            ## Wait a second
            time.sleep(1)